In [ ]:
pip install tensorflow opencv-python-headless fastapi python-telegram-bot reportlab xlsxwriter plotly

In [1]:
import face_recognition
import cv2
import numpy as np
from datetime import datetime, timedelta
import os
import logging
import dlib
from PIL import Image
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import requests
import bz2
import shutil
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, Boolean, LargeBinary, ForeignKey, JSON
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import yaml
import json
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import threading
import queue
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import hashlib
import uuid
import platform
import psutil
from typing import Optional, List, Dict
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Temel yapılandırma
CONFIG = {
    'database': {
        'url': 'sqlite:///face_recognition.db',
        'backup_dir': 'backups'
    },
    'security': {
        'secret_key': uuid.uuid4().hex,
        'token_expiry': 86400,
        'min_password_length': 8
    },
    'gui': {
        'window_title': 'Gelişmiş Yüz Tanıma Sistemi',
        'window_width': 1200,
        'window_height': 800,
        'theme': 'light'
    },
    'camera': {
        'device_id': 0,
        'frame_width': 640,
        'frame_height': 480,
        'fps': 30
    },
    'face_recognition': {
        'tolerance': 0.6,
        'model': 'hog',
        'min_face_size': 20
    },
    'notification': {
        'email': {
            'smtp_server': 'smtp.gmail.com',
            'smtp_port': 587,
            'sender_email': '',
            'sender_password': ''
        },
        'telegram': {
            'bot_token': '',
            'chat_id': ''
        }
    }
}

# Logging ayarları
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('app.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Veritabanı modelleri
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    username = Column(String, unique=True, nullable=False)
    password_hash = Column(String, nullable=False)
    email = Column(String)
    role = Column(String, default='user')
    is_active = Column(Boolean, default=True)
    last_login = Column(DateTime)
    created_at = Column(DateTime, default=datetime.now)

class Person(Base):
    __tablename__ = 'persons'

    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True, nullable=False)
    face_encoding = Column(LargeBinary, nullable=False)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, nullable=False)
    updated_at = Column(DateTime, nullable=True)
    details = Column(JSON)
    email = Column(String)
    phone = Column(String)
    department = Column(String)
    access_level = Column(Integer, default=1)
    last_seen = Column(DateTime)
    photo_path = Column(String)
    
    recognition_logs = relationship("FaceRecognitionLog", back_populates="person")

class FaceRecognitionLog(Base):
    __tablename__ = 'recognition_logs'

    id = Column(Integer, primary_key=True)
    person_id = Column(Integer, ForeignKey('persons.id'))
    confidence_score = Column(Float)
    timestamp = Column(DateTime, nullable=False)
    location = Column(String)
    device_id = Column(String)
    
    person = relationship("Person", back_populates="recognition_logs")

# Yardımcı sınıflar
class AIFeatures:
    def __init__(self):
        self.models = {}
    
    def load_models(self):
        # AI modellerini yükle
        pass
    
    def predict_age_gender(self, face_image):
        return {
            'age': {'value': 25, 'confidence': 0.85},
            'gender': {'value': 'Erkek', 'confidence': 0.92}
        }
    
    def analyze_emotion(self, face_image):
        emotions = {
            'mutlu': 0.8,
            'üzgün': 0.05,
            'kızgın': 0.02,
            'şaşkın': 0.03,
            'nötr': 0.1
        }
        return emotions

class NotificationManager:
    def __init__(self, config):
        self.config = config
    
    def send_email(self, subject, message, to_email):
        try:
            msg = MIMEMultipart()
            msg['From'] = self.config['notification']['email']['sender_email']
            msg['To'] = to_email
            msg['Subject'] = subject
            msg.attach(MIMEText(message, 'plain'))
            
            server = smtplib.SMTP(
                self.config['notification']['email']['smtp_server'],
                self.config['notification']['email']['smtp_port']
            )
            server.starttls()
            server.login(
                self.config['notification']['email']['sender_email'],
                self.config['notification']['email']['sender_password']
            )
            server.send_message(msg)
            server.quit()
            return True
        except Exception as e:
            logger.error(f"E-posta gönderme hatası: {str(e)}")
            return False

class SecurityManager:
    def __init__(self, config):
        self.config = config
        self.blacklist = set()
    
    def check_access(self, person_id, zone_id=None):
        if person_id in self.blacklist:
            return False
        return True
    
    def add_to_blacklist(self, person_id):
        self.blacklist.add(person_id)

class FaceRecognition:
    def __init__(self, db_session):
        self.db_session = db_session
        self.known_face_encodings = []
        self.known_face_names = []
        self.load_known_faces()
        self.ai_features = AIFeatures()

    def load_known_faces(self):
        try:
            persons = self.db_session.query(Person).filter_by(is_active=True).all()
            for person in persons:
                self.known_face_encodings.append(np.frombuffer(person.face_encoding))
                self.known_face_names.append(person.name)
        except Exception as e:
            logger.error(f"Yüzler yüklenirken hata: {str(e)}")

    def add_person(self, image_path, name, details=None):
        try:
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)
            
            if not face_encodings:
                return False
            
            person = Person(
                name=name,
                face_encoding=face_encodings[0].tobytes(),
                created_at=datetime.now(),
                details=details or {}
            )
            
            self.db_session.add(person)
            self.db_session.commit()
            
            self.known_face_encodings.append(face_encodings[0])
            self.known_face_names.append(name)
            
            return True
        except Exception as e:
            self.db_session.rollback()
            logger.error(f"Kişi eklenirken hata: {str(e)}")
            return False

    def recognize_face(self, face_encoding):
        try:
            if not self.known_face_encodings:
                return "Bilinmeyen"
            
            matches = face_recognition.compare_faces(
                self.known_face_encodings,
                face_encoding,
                tolerance=CONFIG['face_recognition']['tolerance']
            )
            
            if True in matches:
                first_match_index = matches.index(True)
                name = self.known_face_names[first_match_index]
                
                # Log kaydı
                person = self.db_session.query(Person).filter_by(name=name).first()
                if person:
                    log = FaceRecognitionLog(
                        person_id=person.id,
                        confidence_score=1.0,
                        timestamp=datetime.now()
                    )
                    self.db_session.add(log)
                    self.db_session.commit()
                
                return name
            
            return "Bilinmeyen"
        except Exception as e:
            logger.error(f"Yüz tanıma hatası: {str(e)}")
            return "Bilinmeyen"

class FaceRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.setup_database()
        self.face_recognizer = FaceRecognition(self.db_session)
        self.notification_manager = NotificationManager(CONFIG)
        self.security_manager = SecurityManager(CONFIG)
        
        self.setup_gui()
        self.start_monitoring()

    def setup_database(self):
        engine = create_engine(CONFIG['database']['url'])
        Base.metadata.create_all(engine)
        Session = sessionmaker(bind=engine)
        self.db_session = Session()

    def setup_gui(self):
        self.root.title(CONFIG['gui']['window_title'])
        self.root.geometry(f"{CONFIG['gui']['window_width']}x{CONFIG['gui']['window_height']}")
        
        self.create_menu()
        self.create_main_frame()
        self.create_status_bar()

    def create_menu(self):
        menubar = tk.Menu(self.root)
        self.root.config(menu=menubar)
        
        # Dosya menüsü
        file_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Dosya", menu=file_menu)
        file_menu.add_command(label="Kişi Ekle", command=self.add_person_dialog)
        file_menu.add_command(label="Veritabanı Yedekle", command=self.backup_database)
        file_menu.add_separator()
        file_menu.add_command(label="Çıkış", command=self.root.quit)
        
        # İşlemler menüsü
        operations_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="İşlemler", menu=operations_menu)
        operations_menu.add_command(label="Kamera Başlat", command=self.start_camera)
        operations_menu.add_command(label="Kamera Durdur", command=self.stop_camera)
        
        # Raporlar menüsü
        reports_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Raporlar", menu=reports_menu)
        reports_menu.add_command(label="Günlük Rapor", command=self.generate_daily_report)
        reports_menu.add_command(label="Haftalık Rapor", command=self.generate_weekly_report)
        
        # Ayarlar menüsü
        settings_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Ayarlar", menu=settings_menu)
        settings_menu.add_command(label="Genel Ayarlar", command=self.show_settings)
        settings_menu.add_command(label="Kamera Ayarları", command=self.show_camera_settings)

    def create_main_frame(self):
        self.main_frame = ttk.Frame(self.root, padding="10")
        self.main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # Kamera görüntüsü
        self.camera_label = ttk.Label(self.main_frame)
        self.camera_label.grid(row=0, column=0, padx=5, pady=5)
        
        # Kontrol paneli
        controls = ttk.Frame(self.main_frame)
        controls.grid(row=1, column=0, padx=5, pady=5)
        
        ttk.Button(controls, text="Kişi Ekle", command=self.add_person_dialog).pack(side=tk.LEFT, padx=5)
        ttk.Button(controls, text="Rapor Oluştur", command=self.generate_daily_report).pack(side=tk.LEFT, padx=5)

    def create_status_bar(self):
        self.status_var = tk.StringVar(value="Hazır")
        status_bar = ttk.Label(self.root, textvariable=self.status_var)
        status_bar.grid(row=2, column=0, sticky=(tk.W, tk.E))

    def start_monitoring(self):
        def monitor():
            while True:
                try:
                    # Sistem durumunu kontrol et
                    cpu_percent = psutil.cpu_percent()
                    memory_percent = psutil.virtual_memory().percent
                    
                    if cpu_percent > 90 or memory_percent > 90:
                        self.notification_manager.send_email(
                            "Sistem Uyarısı",
                            f"Yüksek kaynak kullanımı: CPU: {cpu_percent}%, RAM: {memory_percent}%",
                            CONFIG['notification']['email']['sender_email']
                        )
                    
                    time.sleep(300)  # 5 dakika bekle
                except Exception as e:
                    logger.error(f"İzleme hatası: {str(e)}")
        
        threading.Thread(target=monitor, daemon=True).start()

    def add_person_dialog(self):
        dialog = tk.Toplevel(self.root)
        dialog.title("Kişi Ekle")
        dialog.geometry("400x300")
        
        ttk.Label(dialog, text="İsim:").pack(pady=5)
        name_var = tk.StringVar()
        ttk.Entry(dialog, textvariable=name_var).pack(pady=5)
        
        ttk.Label(dialog, text="Departman:").pack(pady=5)
        dept_var = tk.StringVar()
        ttk.Entry(dialog, textvariable=dept_var).pack(pady=5)
        
        def select_photo():
            file_path = filedialog.askopenfilename(
                filetypes=[("Resim Dosyaları", "*.jpg *.jpeg *.png")]
            )
            if file_path:
                if self.face_recognizer.add_person(
                    file_path,
                    name_var.get(),
                    {'department': dept_var.get()}
                ):
                    messagebox.showinfo("Başarılı", "Kişi başarıyla eklendi!")
                    dialog.destroy()
                else:
                    messagebox.showerror("Hata", "Kişi eklenirken bir hata oluştu!")
        
        ttk.Button(dialog, text="Fotoğraf Seç", command=select_photo).pack(pady=20)

    def generate_daily_report(self):
        try:
            today = datetime.now().date()
            logs = self.db_session.query(FaceRecognitionLog).filter(
                FaceRecognitionLog.timestamp >= today
            ).all()
            
            report_path = f"rapor_{today.strftime('%Y%m%d')}.pdf"
            
            c = canvas.Canvas(report_path, pagesize=letter)
            c.drawString(100, 750, "Günlük Yüz Tanıma Raporu")
            c.drawString(100, 730, f"Tarih: {today.strftime('%d.%m.%Y')}")
            
            total_recognitions = len(logs)
            unique_persons = len(set(log.person_id for log in logs))
            
            c.drawString(100, 700, f"Toplam Tanıma: {total_recognitions}")
            c.drawString(100, 680, f"Tekil Kişi: {unique_persons}")
            
            c.save()
            
            messagebox.showinfo("Başarılı", f"Rapor oluşturuldu: {report_path}")
        except Exception as e:
            messagebox.showerror("Hata", f"Rapor oluşturma hatası: {str(e)}")

def main():
    try:
        root = tk.Tk()
        app = FaceRecognitionApp(root)
        root.mainloop()
    except Exception as e:
        logger.error(f"Uygulama başlatılırken hata: {str(e)}")
        messagebox.showerror("Kritik Hata", str(e))

if __name__ == "__main__":
    main()

C:\Users\msı\AppData\Local\Temp\ipykernel_15808\3356385738.py:89: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
2025-02-02 18:19:59,923 - __main__ - ERROR - Yüzler yüklenirken hata: (sqlite3.OperationalError) no such column: persons.updated_at
[SQL: SELECT persons.id AS persons_id, persons.name AS persons_name, persons.face_encoding AS persons_face_encoding, persons.is_active AS persons_is_active, persons.created_at AS persons_created_at, persons.updated_at AS persons_updated_at, persons.details AS persons_details, persons.email AS persons_email, persons.phone AS persons_phone, persons.department AS persons_department, persons.access_level AS persons_access_level, persons.last_seen AS persons_last_seen, persons.photo_path AS persons_photo_path 
FROM persons 
WHERE persons.is_active = 1]
(Background on this error